In [1]:
from pipeline.utils import load_config
config = load_config()

In [ ]:
from pipeline.data_prep import create_postgres_connection, read_data, create_mysql_connection

In [3]:
p_conn = create_postgres_connection(config)
m_conn = create_mysql_connection(config)

Connected to MySQL database successfully!


In [4]:
res = read_data(p_conn, '"CONSTANT"').iloc[0].to_dict()

In [5]:
from dataclasses import dataclass
from datetime import date
import datetime
@dataclass
class State:
    date: date
    dayofweek: int

In [6]:
state = State(date=res['date'],dayofweek=res['dayofweek'])
state

State(date=datetime.date(2023, 2, 15), dayofweek=1)

In [7]:
mysql_tables = ['traffic_details', 'truck_schedule_data', 'city_weather']
postgres_tables = ['routes_weather']

In [8]:
traffic_df = read_data(m_conn,mysql_tables[0])
truck_schedule_df = read_data(m_conn,mysql_tables[1])
city_weather_df = read_data(m_conn,mysql_tables[2])
route_weather_df = read_data(p_conn,postgres_tables[0])

In [9]:
traffic_df.head(1)

,route_id,date,hour,no_of_vehicles,accident
0,R-ada2a391,2019-01-01,0,669.0,0


In [10]:
truck_schedule_df.head(1)

,truck_id,route_id,departure_date,estimated_arrival,delay
0,30312694,R-b236e347,2019-01-01 07:00:00,2019-01-01 13:13:12.,0


In [11]:
city_weather_df.head(1)

,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
0,C-927ceb5e,2019-01-01,0,30.0,11.0,Light snow,0.0,86,6.0,1019.0,0.0,0.0,0.0,0.0


In [12]:
route_weather_df.head(1)

,route_id,Date,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
0,R-ada2a391,2019-01-01,30.0,11.0,Heavy snow,0.0,90,1.0,1010,0,0,0,0


In [13]:
def filter(df,date,is_route_weather=False):
    if is_route_weather:
        return df[df['Date'] > date]
    else:
        return df[df['date'] > date]

In [14]:
new_df = filter(city_weather_df,datetime.date(2019,2,1))

In [15]:
from evidently.test_suite import TestSuite
from evidently.tests.base_test import generate_column_tests
from evidently.tests import *

In [ ]:
def run_data_quality_check(reference_data, current_data):
    tests = TestSuite(tests=[
        TestNumberOfRowsWithMissingValues(),
        TestNumberOfDuplicatedRows(),
        TestNumberOfDuplicatedColumns(),
        TestColumnsType()
    ])
    tests.run(reference_data=reference_data, current_data=current_data)
    return tests, tests.json()

In [19]:
tests, tests_json = run_data_quality_check(city_weather_df,new_df)

In [53]:
def assert_quality_passed(suite):
    failed_tests = [t for t in suite['tests'] if t['status'] == "FAIL"]

    if failed_tests:
        names = [t.name for t in failed_tests]
        raise ValueError(f"❌ Data Quality Check Failed: {names}")
    print("✅ All Data Quality Tests Passed")

In [54]:
assert_quality_passed(json.loads(tests_json))

✅ All Data Quality Tests Passed
